## Extract data from the Querki  
It is bascially rule references for Spirit island board game.

In [1]:
from scripts import Scraper


base_url = "https://querki.net/u/darker/spirit-island-faq/#!"
home_suffix = "spirit-island-faq"

xpaths = {
    "errata": '//*[@id="_root"]/div/div[2]/div/div/div[3]/div/div[3]/ul[1]',
    "tags": '//*[@id="_root"]/div/div[2]/div/div/div[3]/div/div[3]/ul[2]',
    "content": '//*[@id="_root"]/div/div[2]/div/div/div[3]/div/div[3]',
}

In [2]:
scraper = Scraper()
url = base_url + home_suffix
errata = scraper.scrape(url, xpaths["errata"])

In [4]:
errata_urls = [
    item.replace(" ", "-")
    for item in errata.text.split("\n")
]

In [5]:
url = base_url + errata_urls[0]
content = scraper.scrape(url, xpaths["content"])

In [6]:
print(content.text)

Here are instructions if you got a playmat without an instruction sheet:
What you need to know to use the playmat (most of which boils down to "it works like you're using the boards"):
The island is still divided into "boards" for game-effect purposes. The borders between boards are a bit thicker than the other land boundaries.
You make use of one "board" per player, just as shown in the core rules. You can put Fear markers into out-of-play lands that border in-play lands, cover over unused boards with decks/supplies, and/or make an actual border-line with unused Presence / pebbles / paperclips - whatever you find easiest.
The Ocean on each "board" is distinct, just like if you were playing without the playmat. (This is relevant for Ocean's Hungry Grasp.)
The northern and southern edges of the island are NOT considered Coastal, just like if you were playing without the playmat. The rocky cliffs which make them inaccessible from the Ocean can be seen along the southern border; the persp

In [7]:
scraper.terminate()

## Retrieve more on the errata content

In [19]:
import time 


def get_content(element, char_for_space: str):
    content = {}
    for item in element.text.split("\n"):
        try:
            url = base_url + item.replace(" ", char_for_space)
            content[item] = scraper.scrape(url, xpaths["content"]).text
            time.sleep(2)
        except:
            url = base_url + "what-items-are-triggered-actions"
            content[item] = scraper.scrape(url, xpaths["content"]).text
    return content

In [20]:
scraper = Scraper()
url = base_url + home_suffix
errata = scraper.scrape(url, xpaths["errata"])
errata_content = get_content(errata, "-")
scraper.terminate()

In [21]:
errata_content

{'Playmat instructions': 'Here are instructions if you got a playmat without an instruction sheet:\nWhat you need to know to use the playmat (most of which boils down to "it works like you\'re using the boards"):\nThe island is still divided into "boards" for game-effect purposes. The borders between boards are a bit thicker than the other land boundaries.\nYou make use of one "board" per player, just as shown in the core rules. You can put Fear markers into out-of-play lands that border in-play lands, cover over unused boards with decks/supplies, and/or make an actual border-line with unused Presence / pebbles / paperclips - whatever you find easiest.\nThe Ocean on each "board" is distinct, just like if you were playing without the playmat. (This is relevant for Ocean\'s Hungry Grasp.)\nThe northern and southern edges of the island are NOT considered Coastal, just like if you were playing without the playmat. The rocky cliffs which make them inaccessible from the Ocean can be seen alo

## Retrieve content from tags

In [22]:
import time 


def get_content(element, char_for_space: str):
    content = {}
    for item in element.text.split("\n"):
        url = base_url + item.replace(" ", char_for_space)
        content[item] = scraper.scrape(url, xpaths["content"]).text
        time.sleep(2)
    return content

In [23]:
scraper = Scraper()
url = base_url + home_suffix
tags = scraper.scrape(url, xpaths["tags"])
tags_content = get_content(tags, "+")
scraper.terminate()

## Combine the content and convert to pandas dataframe

In [24]:
all_content = [
    [item, content, "errata"]
    for item, content in errata_content.items()
]

all_content += [
    [item, content, "tags"]
    for item, content in tags_content.items()
]

In [26]:
from pandas import DataFrame


df = DataFrame(all_content, columns=["topic", "content", "category"])
df.head(3)

,topic,content,category
0,Playmat instructions,Here are instructions if you got a playmat wit...,errata
1,Summary of Action rules,"This page is so folks can quickly reference ""w...",errata
2,New Nomenclature Roundup,Older game items - particularly Event Cards - ...,errata


In [27]:
df.to_csv("../data/si_faq.csv", index=False)